## This program lets you get part descriptions, find all the assemblies that the part is in and to get the bom of the assembly. It could also be used to find the parts by serching in the descriptions.

In [2]:
# pip install pyodbc
# pip install pandas
# pip install tabulate


In [1]:
from tabulate import tabulate
def display_table(data_frame):
    print(tabulate(data_frame, headers='keys', tablefmt='psql'))

In [2]:
## For shop orders with history
import pyodbc
from tabulate import tabulate
import pandas as pd
import datetime

max_connection = pyodbc.connect("Driver={SQL Server}; Server=main2; Database=ExactMAXITC; Uid=reports; Pwd=reports;")
cursor = max_connection.cursor()

def getorderdesc(prt_num):
    query = """
        SELECT
            ORDNUM_10, 
            PRTNUM_10, 
            CURDUE_10, 
            ORGDUE_10, 
            CURQTY_10, 
            STATUS_10, 
            COST_10
        FROM [ExactMAXITC].[dbo].Order_Master
        WHERE PRTNUM_10 = ?
    """

    cursor.execute(query, prt_num)
    partdesc = cursor.fetchall() 
    return(partdesc)

df = pd.DataFrame(columns=['Order No', 'Part No', 'Current Due Date', 'Original Due Date', 'Quantinty', 'Status', 'Cost'])
desc = getorderdesc("200114")
for i in desc:
    mylist = list(i)
    mylist[0] = mylist[0].strip()
    mylist[1] = mylist[1].strip()
    mylist[2] = str(mylist[2]).split()[0]
    mylist[3] = str(mylist[3]).split()[0]
    df.loc[len(df)] = mylist

print(tabulate(df, headers='keys', tablefmt='psql'))

+----+------------+-----------+--------------------+---------------------+-------------+----------+--------+
|    |   Order No |   Part No | Current Due Date   | Original Due Date   |   Quantinty |   Status |   Cost |
|----+------------+-----------+--------------------+---------------------+-------------+----------+--------|
|  0 |   70001618 |    200114 | 1998-02-26         | 1998-02-26          |          10 |        4 |  0.325 |
|  1 |   70002255 |    200114 | 1998-04-13         | 1998-04-13          |          30 |        4 |  0.33  |
|  2 |   70004984 |    200114 | 1999-06-11         | 1999-06-11          |          16 |        4 |  3.25  |
|  3 |   70022156 |    200114 | 2001-03-19         | 2001-03-19          |          20 |        4 |  0.33  |
|  4 |   70023124 |    200114 | 2001-11-14         | 2001-11-14          |          14 |        4 |  3.45  |
|  5 |   70037673 |    200114 | 2007-07-09         | 2007-07-09          |          50 |        4 |  0.387 |
|  6 |   70038786 |

In [3]:
## For shop orders
import pyodbc
from tabulate import tabulate
import pandas as pd
import datetime

max_connection = pyodbc.connect("Driver={SQL Server}; Server=main2; Database=ExactMAXITC; Uid=reports; Pwd=reports;")
cursor = max_connection.cursor()

def getorderdesc(prt_num):
    query = """
        SELECT
            ORDNUM_10, 
            PRTNUM_10, 
            CURDUE_10, 
            ORGDUE_10, 
            CURQTY_10, 
            STATUS_10, 
            COST_10
        FROM [ExactMAXITC].[dbo].Order_Master
        WHERE PRTNUM_10 = ?
    """

    cursor.execute(query, prt_num)
    partdesc = cursor.fetchall()[-1]    ## remove [-1] to get previous data order for the part number
    # partdesc = cursor.fetchall() 
    stringdesc = list(map(str, partdesc))
    formatdesc = list(map(str.strip, stringdesc))
    return(formatdesc)

def orderdesc(prt_num):
    status = {"1":"Planned", "2":"Approved", "3":"Released"}
    formatdesc = getorderdesc(prt_num)
    print(f"Order Number: {formatdesc[0]}")
    print(f"Part Number: {formatdesc[1]}")
    print(f"Current Due Date: {formatdesc[2].split()[0]}")
    print(f"Original Due Date: {formatdesc[3].split()[0]}")
    print(f"Order Quantity: {formatdesc[4]}")
    print(f"Status: {status[formatdesc[5]]}")
    print(f"Cost: {formatdesc[6]}")

orderdesc("0401473")


Order Number: 50056035
Part Number: 0401473
Current Due Date: 2023-11-03
Original Due Date: 2023-11-03
Order Quantity: 1.0
Status: Released
Cost: 0.0


In [9]:
## For Part details
import pyodbc
from tabulate import tabulate
import pandas as pd
import datetime

max_connection = pyodbc.connect("Driver={SQL Server}; Server=main2; Database=ExactMAXITC; Uid=reports; Pwd=reports;")
cursor = max_connection.cursor()

def getpartdesc(prt_num):
    query = """
        SELECT
            PRTNUM_01,
            PMDES1_01,
            PMDES2_01,
            COST_01,
            CURREV_01, 
            UDFKEY_01,
            UDFREF_01,
            VIEWER_01
        FROM [ExactMAXITC].[dbo].Part_Master
        WHERE PRTNUM_01 = ?
    """
    cursor.execute(query, prt_num)
    partdesc = cursor.fetchone()
    stringdesc = list(map(str, partdesc))
    formatdesc = list(map(str.strip, stringdesc))
    formatdesc[3] = "${:,}".format(float(formatdesc[3]))
    print(formatdesc)
    return formatdesc

def part_desc(prt_num):
    description = ['Part_Number', 'Description 1', 'Description 2', 'Cost/Unit', 'Current Revision', 'UDF_Key', 'UDF_Ref'] 
    formatdesc = getpartdesc(prt_num) 
    for i, j in (zip(description, formatdesc)):
        print(f"{i}: {j}")

part_desc("0401473")

Part_Number: 0401473
Description 1: CHUCK 6500, 3 PIN W/SANDING FLAT
Description 2: CPM3V - FF ONLY
Cost/Unit: $0.0
Current Revision: 
UDF_Key: 
UDF_Ref: NOTES


In [5]:
## For parent parts and their details
import pyodbc
from tabulate import tabulate
import pandas as pd
import datetime

max_connection = pyodbc.connect("Driver={SQL Server}; Server=main2; Database=ExactMAXITC; Uid=reports; Pwd=reports;")
cursor = max_connection.cursor()

def getpartdesc(prt_num):
    query = """
        SELECT
            PRTNUM_01,
            PMDES1_01,
            PMDES2_01,
            COST_01,
            CURREV_01, 
            UDFKEY_01,
            UDFREF_01
        FROM [ExactMAXITC].[dbo].Part_Master
        WHERE PRTNUM_01 = ?
    """
    cursor.execute(query, prt_num)
    partdesc = cursor.fetchone()
    stringdesc = list(map(str, partdesc))
    formatdesc = list(map(str.strip, stringdesc))
    formatdesc[3] = "${:,}".format(float(formatdesc[3]))
    return formatdesc

def parent_desc(prt_num):
    parentprts = sorted(cursor.execute("SELECT PARPRT_02 FROM Product_Structure WHERE COMPRT_02= ?", prt_num).fetchall())
    parentprtlist = pd.DataFrame(columns=('Parent_Part', 'Description 1', 'Description 2', 'Cost/Unit', 'Current Revision', 'UDF_Key', 'UDF_Ref'))
    for i in parentprts:
        formatdesc = getpartdesc(i[0])
        parentprtlist.loc[len(parentprtlist)] = formatdesc
    parentprtlist.index += 1
    return parentprtlist

display_table(parent_desc("0401473"))


NameError: name 'display_table' is not defined

In [6]:
## For BOM of parts
import pyodbc
from tabulate import tabulate
import pandas as pd
import datetime

max_connection = pyodbc.connect("Driver={SQL Server}; Server=main2; Database=ExactMAXITC; Uid=reports; Pwd=reports;")
cursor = max_connection.cursor()

def getpartdesc(prt_num):
    query = """
        SELECT
            PRTNUM_01,
            PMDES1_01,
            PMDES2_01,
            COST_01,
            CURREV_01, 
            UDFKEY_01,
            UDFREF_01
        FROM [ExactMAXITC].[dbo].Part_Master
        WHERE PRTNUM_01 = ?
    """
    cursor.execute(query, prt_num)
    partdesc = cursor.fetchone()
    stringdesc = list(map(str, partdesc))
    formatdesc = list(map(str.strip, stringdesc))
    formatdesc[3] = "${:,}".format(float(formatdesc[3]))
    return formatdesc

def bom_desc(prt_num):
    Bomprts = list(cursor.execute("SELECT COMPRT_02, EFFDTE_02, QTYPER_02, QTYCDE_02 FROM Product_Structure WHERE PARPRT_02 = ?", prt_num).fetchall())
    Bomprtlist = pd.DataFrame(columns=('Part id', 'Description 1', 'Effectivity', 'QTY', 'QC', 'Description 2', 'Cost/Unit', 'Rev', 'UDF_Key', 'UDF_Ref'))
    for bompart in Bomprts:
        dt = str(bompart[1]).split(' ')
        qty = str(bompart[2]).strip()
        if len(dt) != 1:
            dt = dt[0].split('-')
            date = f"{dt[1]}/{dt[2]}/{dt[0]}"
            i = [date, qty, bompart[3]]
        else:
            i = ['', qty, bompart[3]]
        formatdesc = getpartdesc(bompart[0])
        Bomprtlist.loc[len(Bomprtlist)] = formatdesc[0:2] + i + formatdesc[2:]
    Bomprtlist = Bomprtlist.sort_values(by=['Part id']).reset_index(drop=True)
    Bomprtlist.index += 1
    formatdesc = getpartdesc(prt_num)
    Bomprtlist.loc[0] = [f"{formatdesc[0]} (Parent)"] + formatdesc[1:2] + ['-','-','-'] + formatdesc[2:]
    Bomprtlist = Bomprtlist.sort_index()
    return Bomprtlist

display_table(bom_desc("002450189C"))

+----+---------------------+-----------------------------------+---------------+--------+------+---------------------------+-------------+-------+-----------------+---------------------------+
|    | Part id             | Description 1                     | Effectivity   | QTY    | QC   | Description 2             | Cost/Unit   | Rev   | UDF_Key         | UDF_Ref                   |
|----+---------------------+-----------------------------------+---------------+--------+------+---------------------------+-------------+-------+-----------------+---------------------------|
|  0 | 002450189C (Parent) | POP-UP GRAM FORCE MODULE HIGH RES | -             | -      | -    | .010" BEAM                | $1,103.6538 |       |                 | NOTES                     |
|  1 | 002450188B          | PCBA POP-UP G/F PRE-AMP           | 06/04/2015    | 1.0    | U    |                           | $188.4195   |       |                 |                           |
|  2 | 0400620             | BLOCK 

In [11]:
## For Pats by description
import pyodbc
from tabulate import tabulate
import pandas as pd
import datetime

max_connection = pyodbc.connect("Driver={SQL Server}; Server=main2; Database=ExactMAXITC; Uid=reports; Pwd=reports;")
cursor = max_connection.cursor()

def getpartdesc(prt_num):
    query = """
        SELECT
            PRTNUM_01,
            PMDES1_01,
            PMDES2_01,
            COST_01,
            CURREV_01, 
            UDFKEY_01,
            UDFREF_01
        FROM [ExactMAXITC].[dbo].Part_Master
        WHERE PRTNUM_01 = ?
    """
    cursor.execute(query, prt_num)
    partdesc = cursor.fetchone()
    stringdesc = list(map(str, partdesc))
    formatdesc = list(map(str.strip, stringdesc))
    formatdesc[3] = "${:,}".format(float(formatdesc[3]))
    return formatdesc

def find_by_desc(query):
    desc = list(cursor.execute("SELECT PRTNUM_01, PMDES1_01, PMDES2_01, COST_01, CURREV_01, UDFKEY_01, UDFREF_01, CreationDate FROM Part_Master")) 
    strin = lambda x: str(x).strip()
    desc = [[strin(i[0]), strin(i[1]), strin(i[2]), strin(i[3]), strin(i[4]), strin(i[5]), strin(i[6]), strin(i[7]).split()[0].rstrip()] for i in desc if query.lower() in i[0].lower() or query.lower() in i[1].lower() or query.lower() in i[2].lower() or query.lower() in i[6].lower()]
    description = pd.DataFrame(desc, columns=['Part_Id', 'Description 1', 'Description 2', 'Cost/Unit', 'Current Revision', 'UDF_Key', 'UDF_Ref', 'CreationDate'])
    return description

display_table(find_by_desc("tooling ball"))


+----+------------------------------+--------------------------------+---------------------------+-------------+--------------------+---------------+---------------------------+----------------+
|    | Part_Id                      | Description 1                  | Description 2             |   Cost/Unit | Current Revision   | UDF_Key       | UDF_Ref                   | CreationDate   |
|----+------------------------------+--------------------------------+---------------------------+-------------+--------------------+---------------+---------------------------+----------------|
|  0 | 002450147A                   | TOOLING BALL SEAL-SHT-OBS      | 3072                      |     421.456 |                    |               |                           | 2011-04-20     |
|  1 | 002450147B                   | TOOLING BALL SEAL-LNG-OBS      | 3072 S9000, J973          |     402.944 |                    |               |                           | 2011-04-20     |
|  2 | 0400203           

#### The below part is for class it is not versatile we need to make changes to all the parts if you want to add a extra column so went with the above option. you can also call the functions as given one you click on run all.

orderdesc("0400891") &emsp;&emsp;&emsp; \# Gives Order details for the part

part_desc("0400891") &emsp;&emsp;&emsp; \# Gives part descriptions and other details

display_table(parent_desc("0400891")) &emsp; \# Gives description of all the parts the current part is a part of

display_table(bom_desc("0400891")) &emsp;&emsp; \# Gives the Bill of Materials(BOM) for the given part

display_table(find_by_desc("0400891")) &emsp; \# Gives all the available parts with the given description



In [8]:
# orderdesc("0400891")
part_desc("0424846")
# display_table(parent_desc("382M24MDXS"))
# display_table(bom_desc("0024706A"))
# display_table(find_by_desc("0401408"))

Part_Number: 0424846
Description 1: LEVELING FOOT & NUT ASSY
Description 2: 
Released on: $52.1637
Cost/Unit: 
Current Revision: 
UDF_Key: 


In [9]:
# import pyodbc
# import pandas as pd
# from tabulate import tabulate

# max_connection = pyodbc.connect("Driver={SQL Server}; Server=main2; Database=ExactMAXITC; Uid=reports; Pwd=reports;")

# class ExactMax:
#     def __init__(self, prt_num=None):
#         self.prt_num = prt_num
#         self.cursor = max_connection.cursor()

#     def getpartdesc(self, prt_num):
#         query = """
#             SELECT
#                 PRTNUM_01,
#                 PMDES1_01,
#                 PMDES2_01,
#                 COST_01,
#                 CURREV_01, 
#                 UDFKEY_01,
#                 UDFREF_01
#             FROM [ExactMAXITC].[dbo].Part_Master
#             WHERE PRTNUM_01 = ?
#         """
#         self.cursor.execute(query, prt_num)
#         partdesc = self.cursor.fetchone()
#         stringdesc = list(map(str, partdesc))
#         formatdesc = list(map(str.strip, stringdesc))
#         formatdesc[3] = "${:,}".format(float(formatdesc[3]))
#         return formatdesc

#     def part_desc(self):
#         description = ['Part_Number', 'Description 1', 'Description 2', 'Released on', 'Cost/Unit', 'Current Revision', 'UDF_Key', 'UDF_Ref'] 
#         formatdesc = self.getpartdesc(self.prt_num) 
#         return dict(zip(description, formatdesc))
        
#     def parent_desc(self):
#         parentprts = sorted(self.cursor.execute("SELECT PARPRT_02 FROM Product_Structure WHERE COMPRT_02= ?", self.prt_num).fetchall())
#         parentprtlist = pd.DataFrame(columns=('Parent_Part', 'Description 1', 'Description 2', 'Cost/Unit', 'Current Revision', 'UDF_Key', 'UDF_Ref'))
#         for i in parentprts:
#             formatdesc = self.getpartdesc(i[0])
#             parentprtlist.loc[len(parentprtlist)] = formatdesc
#         parentprtlist.index += 1
#         return parentprtlist

#     def bom_desc(self):
#         Bomprts = list(self.cursor.execute("SELECT COMPRT_02, EFFDTE_02, QTYPER_02, QTYCDE_02 FROM Product_Structure WHERE PARPRT_02 = ?", self.prt_num).fetchall())
#         Bomprtlist = pd.DataFrame(columns=('Part id', 'Description 1', 'Effectivity', 'QTY', 'QC', 'Description 2', 'Cost/Unit', 'Rev', 'UDF_Key', 'UDF_Ref'))
#         for bompart in Bomprts:
#             dt = str(bompart[1]).split(' ')
#             qty = str(bompart[2]).strip()
#             if len(dt) != 1:
#                 dt = dt[0].split('-')
#                 date = f"{dt[1]}/{dt[2]}/{dt[0]}"
#                 i = [date, qty, bompart[3]]
#             else:
#                 i = ['', qty, bompart[3]]
#             formatdesc = self.getpartdesc(bompart[0])
#             Bomprtlist.loc[len(Bomprtlist)] = formatdesc[0:2] + i + formatdesc[2:]
#         Bomprtlist = Bomprtlist.sort_values(by=['Part id']).reset_index(drop=True)
#         Bomprtlist.index += 1
#         formatdesc = self.getpartdesc(self.prt_num)
#         Bomprtlist.loc[0] = [f"{formatdesc[0]} (Parent)"] + formatdesc[1:2] + ['-','-','-'] + formatdesc[2:]
#         Bomprtlist = Bomprtlist.sort_index()
#         return Bomprtlist

#     def find_by_desc(self, query):
#         desc = list(self.cursor.execute("SELECT PRTNUM_01, PMDES1_01, PMDES2_01, COST_01, CURREV_01, UDFKEY_01, UDFREF_01, CreationDate FROM Part_Master")) 
#         strin = lambda x: str(x).strip()
#         desc = [[strin(i[0]), strin(i[1]), strin(i[2]), strin(i[3]), strin(i[4]), strin(i[5]), strin(i[6]), strin(i[7]).split()[0].rstrip()] for i in desc if query.lower() in i[0].lower() or query.lower() in i[1].lower() or query.lower() in i[2].lower() or query.lower() in i[6].lower()]
#         description = pd.DataFrame(desc, columns=['Part_Id', 'Description 1', 'Description 2', 'Cost/Unit', 'Current Revision', 'UDF_Key', 'UDF_Ref', 'CreationDate'])
#         return description


### Usage

In [12]:
# from tabulate import tabulate

# def display_table(data_frame):
#     print(tabulate(data_frame, headers='keys', tablefmt='psql'))

# def display_description(description):
#     for i,j in zip(description.keys(), description.values()):
#         print(f"{i}: {j}")

# # description = ExactMax('002450008A').part_desc()
# # display_description(description)

# # parent = ExactMax('0401564').parent_desc()
# # display_table(parent)

# # bom = ExactMax('002450000D').bom_desc()
# # # bom = bom[bom['Part id'].str[:2] == "04"]
# # display_table(bom)

# # found_parts = ExactMax().find_by_desc('GLASS STRIP X ENCODER')
# # display_table(found_parts)


In [13]:
# parent = ExactMax('002450008A').bom_desc()
# display_table(parent)

# gramforce = parent[['Part id', 'Description 1']]

In [14]:
## To get all the tables available and stuff
# cursor.execute("SELECT * FROM sys.tables")
# tables = cursor.fetchall()
# for i in tables: 
#     print(i)

In [15]:
# ## For Part details
# import pyodbc
# from tabulate import tabulate
# from tkinter import *

# max_connection = pyodbc.connect("Driver={SQL Server}; Server=main2; Database=ExactMAXITC; Uid=reports; Pwd=reports;")
# cursor = max_connection.cursor()

# def getpartdesc(prt_num):
#     query = """
#         SELECT
#             PRTNUM_01,
#             PMDES1_01,
#             PMDES2_01,
#             COST_01,
#             CURREV_01, 
#             UDFKEY_01,
#             UDFREF_01
#         FROM [ExactMAXITC].[dbo].Part_Master
#         WHERE PRTNUM_01 = ?
#     """
#     cursor.execute(query, prt_num)
#     partdesc = cursor.fetchone()
#     stringdesc = list(map(str, partdesc))
#     formatdesc = list(map(str.strip, stringdesc))
#     formatdesc[3] = "${:,}".format(float(formatdesc[3]))
#     return formatdesc

# def part_desc(prt_num):
#     description = ['Part_Number', 'Description 1', 'Description 2', 'Cost/Unit', 'Current Revision', 'UDF_Key', 'UDF_Ref'] 
#     formatdesc = getpartdesc(prt_num) 
#     output_txt = ""
#     for i, j in (zip(description, formatdesc)):
#         output_txt += f"{i}: {j}\n"
#     return output_txt

# root = Tk()
# partname = root.clipboard_get()
# output_txt = part_desc("0401473")
# root.title("Part Details")
# w = Label(root, text = output_txt)
# w.pack()
# root.mainloop()
